###### Imports and Settings

In [1]:
import folium
from folium import plugins
from folium import Map
from folium.map import Layer, FeatureGroup,LayerControl,Marker
from folium.plugins import MarkerCluster,FeatureGroupSubGroup,Fullscreen
import geopandas as gpd
import pandas as pd
import branca
import branca.colormap as cm
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

# Map Clusters  

Here the shapefile of the places coming from the data prep for all places in the US along with their clusters as established by the K-Means clustering algorithm. Using Folium, interactive maps are created to visualize where these peer communities are located.

In [2]:
geo = gpd.read_file('../data/geo/ALLPLACESCLUSTER.shp')

In [3]:
geo.head(2)

,GEOID,NAME,NAMELSAD,SQMILES,StateFIPS,PlaceFIPS,09pop,14pop,19pop,hhsize,drive,ST%,ST,LT%,LT,State,19density,14density,09density,STDensity,ST%Density,LTDensity,LT%Density,Place,CBSA,14permits,19permits,permreal,perm%,cluster,geometry
0,0115136,Clanton,Clanton city,22.84,01,15136,8689.0,8672.0,8700.0,2.44,31.1,0.32,28.0,0.13,11.0,Alabama,380.91,379.68,380.43,1.23,0.32,0.48,0.13,Clanton,13820,23.4,26.4,3.0,12.82,0,"POLYGON ((-86.68405 32.83101, -86.68383 32.831..."
1,0163336,Rainsville,Rainsville city,20.62,01,63336,4898.0,4984.0,5041.0,2.66,21.5,1.14,57.0,2.92,143.0,Alabama,244.47,241.71,237.54,2.76,1.14,6.93,2.92,Rainsville,99999,11.2,23.2,12.0,107.14,1,"POLYGON ((-85.89968 34.51085, -85.89943 34.511..."


In [4]:
print(geo.crs)

epsg:4269


In [5]:
#copy to new geodataframe of points
points = geo.copy()
#change the geometry
points = points.to_crs('EPSG:4269')
points.geometry = points['geometry'].centroid
#set crs to be the same
points.crs = geo.crs
points.head()

C:\Users\jmccall\AppData\Local\Temp/ipykernel_19156/3514061553.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points.geometry = points['geometry'].centroid


,GEOID,NAME,NAMELSAD,SQMILES,StateFIPS,PlaceFIPS,09pop,14pop,19pop,hhsize,drive,ST%,ST,LT%,LT,State,19density,14density,09density,STDensity,ST%Density,LTDensity,LT%Density,Place,CBSA,14permits,19permits,permreal,perm%,cluster,geometry
0,0115136,Clanton,Clanton city,22.84,01,15136,8689.0,8672.0,8700.0,2.44,31.1,0.32,28.0,0.13,11.0,Alabama,380.91,379.68,380.43,1.23,0.32,0.48,0.13,Clanton,13820,23.4,26.4,3.0,12.82,0,POINT (-86.62298 32.84392)
1,0163336,Rainsville,Rainsville city,20.62,01,63336,4898.0,4984.0,5041.0,2.66,21.5,1.14,57.0,2.92,143.0,Alabama,244.47,241.71,237.54,2.76,1.14,6.93,2.92,Rainsville,99999,11.2,23.2,12.0,107.14,1,POINT (-85.84353 34.49385)
2,0171976,Spanish Fort,Spanish Fort city,30.22,01,71976,6081.0,7357.0,8601.0,2.62,25.0,16.91,1244.0,41.44,2520.0,Alabama,284.61,243.45,201.22,41.16,16.91,83.39,41.44,Spanish Fort,19300,19.8,138.0,118.2,596.97,5,POINT (-87.86500 30.72350)
3,0171976,Spanish Fort,Spanish Fort city,30.22,01,71976,6081.0,7357.0,8601.0,2.62,25.0,16.91,1244.0,41.44,2520.0,Alabama,284.61,243.45,201.22,41.16,16.91,83.39,41.44,Spanish Fort,99999,46.2,0.0,-46.2,-100.00,5,POINT (-87.86500 30.72350)
4,0116768,Columbiana,Columbiana city,17.30,01,16768,3892.0,4198.0,4481.0,2.20,26.8,6.74,283.0,15.13,589.0,Alabama,259.02,242.66,224.97,16.36,6.74,34.05,15.14,Columbiana,13820,2.6,51.2,48.6,1869.23,1,POINT (-86.61141 33.19301)


In [6]:
points['lon'] = points.geometry.x
points['lat'] = points.geometry.y

In [7]:
points.head()

,GEOID,NAME,NAMELSAD,SQMILES,StateFIPS,PlaceFIPS,09pop,14pop,19pop,hhsize,drive,ST%,ST,LT%,LT,State,19density,14density,09density,STDensity,ST%Density,LTDensity,LT%Density,Place,CBSA,14permits,19permits,permreal,perm%,cluster,geometry,lon,lat
0,0115136,Clanton,Clanton city,22.84,01,15136,8689.0,8672.0,8700.0,2.44,31.1,0.32,28.0,0.13,11.0,Alabama,380.91,379.68,380.43,1.23,0.32,0.48,0.13,Clanton,13820,23.4,26.4,3.0,12.82,0,POINT (-86.62298 32.84392),-86.622979,32.843918
1,0163336,Rainsville,Rainsville city,20.62,01,63336,4898.0,4984.0,5041.0,2.66,21.5,1.14,57.0,2.92,143.0,Alabama,244.47,241.71,237.54,2.76,1.14,6.93,2.92,Rainsville,99999,11.2,23.2,12.0,107.14,1,POINT (-85.84353 34.49385),-85.843529,34.493854
2,0171976,Spanish Fort,Spanish Fort city,30.22,01,71976,6081.0,7357.0,8601.0,2.62,25.0,16.91,1244.0,41.44,2520.0,Alabama,284.61,243.45,201.22,41.16,16.91,83.39,41.44,Spanish Fort,19300,19.8,138.0,118.2,596.97,5,POINT (-87.86500 30.72350),-87.865005,30.723497
3,0171976,Spanish Fort,Spanish Fort city,30.22,01,71976,6081.0,7357.0,8601.0,2.62,25.0,16.91,1244.0,41.44,2520.0,Alabama,284.61,243.45,201.22,41.16,16.91,83.39,41.44,Spanish Fort,99999,46.2,0.0,-46.2,-100.00,5,POINT (-87.86500 30.72350),-87.865005,30.723497
4,0116768,Columbiana,Columbiana city,17.30,01,16768,3892.0,4198.0,4481.0,2.20,26.8,6.74,283.0,15.13,589.0,Alabama,259.02,242.66,224.97,16.36,6.74,34.05,15.14,Columbiana,13820,2.6,51.2,48.6,1869.23,1,POINT (-86.61141 33.19301),-86.611415,33.193008


In [8]:
points.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       90 non-null     object  
 1   NAME        90 non-null     object  
 2   NAMELSAD    90 non-null     object  
 3   SQMILES     90 non-null     float64 
 4   StateFIPS   90 non-null     object  
 5   PlaceFIPS   90 non-null     object  
 6   09pop       90 non-null     float64 
 7   14pop       90 non-null     float64 
 8   19pop       90 non-null     float64 
 9   hhsize      90 non-null     float64 
 10  drive       90 non-null     float64 
 11  ST%         90 non-null     float64 
 12  ST          90 non-null     float64 
 13  LT%         90 non-null     float64 
 14  LT          90 non-null     float64 
 15  State       90 non-null     object  
 16  19density   90 non-null     float64 
 17  14density   90 non-null     float64 
 18  09density   90 non-null     float64 
 19  ST

Set initial color return function for all clusters represented by distinct diverging colorscale.

In [9]:
steppt = cm.StepColormap(
    ['#bbf876', '#982f36', '#a3a3a3', '#165f97', '#79accc', '#fc8ee8', '#583768', '#ffa500'],
    vmin=0, vmax=8-1,
    caption='Cluster of Place')
steppt

Initialize full screen plug-in.

In [10]:
fullscr = plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True)

Initialize minimap plug-in.

In [11]:
minimap = plugins.MiniMap()

In [12]:
# for grp_name, df_grp in points.groupby('cluster'):
#     feature_group = folium.FeatureGroup(grp_name)
#     for row in df_grp.itertuples():
#         folium.Marker(location=[row.Latitude, row.Longitude]).add_to(feature_group)
#     feature_group.add_to(m)

# folium.LayerControl().add_to(m)

In [13]:
m = folium.Map(location = [52.610002, -114.625274], 
                      tiles = 'cartodbpositron', zoom_start = 4, control_scale = True)
m.add_child(steppt)
m.add_child(fullscr)
m.add_child(minimap)

for i in range(0,len(points)):
    cluster_num = points['cluster'].iloc[i]
    if cluster_num == 0:
        color = 'lightgreen'
    if cluster_num == 1:
        color = 'darkred'
    if cluster_num == 2:
        color = 'lightgray'
    if cluster_num == 3:
        color = 'darkblue'
    if cluster_num == 4:
        color = 'cadetblue'
    if cluster_num == 5:
        color = 'pink'
    if cluster_num == 6:
        color = 'darkpurple'
    if cluster_num == 7:
        color = 'orange'
        
    cluster_num = points['cluster'].iloc[i]
    if cluster_num == 0:
        cluster = '0'
    if cluster_num == 1:
        cluster = '1'
    if cluster_num == 2:
        cluster = '2'
    if cluster_num == 3:
        cluster = '3'
    if cluster_num == 4:
        cluster = '4'
    if cluster_num == 5:
        cluster = '5'
    if cluster_num == 6:
        cluster = '6'
    if cluster_num == 7:
        cluster = '7'
    
    loc = [points['lat'].iloc[i], points['lon'].iloc[i]]
    labels = points['NAME'].iloc[i] + ", " + points['State'].iloc[i] #label that is the name of the place
    icon = folium.Icon(color=color, icon='asterisk', prefix='fa')
    
    icon_number = plugins.BeautifyIcon(
    border_color=color,
    text_color=color,
    number=cluster,
    inner_icon_style="margin-top:0;")
    
    markers = folium.Marker(location = loc, popup=labels,
                 icon=icon_number).add_to(m)
    
#     #Messing with featuregroupsubgroup
#     fg = folium.FeatureGroup(name = 'Clusters')    .add_to(m)                      # Main group
#     g1 = folium.plugins.FeatureGroupSubGroup(fg, 'Cluster 1').add_to(m)  # First subgroup of fg
#     g2 = folium.plugins.FeatureGroupSubGroup(fg, 'Cluster 2').add_to(m)  # Second subgroup of fg

#     if points.loc[points['cluster'] == 1]:
#         folium.Marker([points['lat'].iloc[i], points['lon'].iloc[i]]).add_to(g1)

#folium.LayerControl(collapsed = False).add_to(m)

m

Now experimenting with featuregroup subgroup...

In [14]:
#create a cluster dict by looping through the dataset and populate the dict with each subset
#instantiate empty dictionary to store subset
cluster_dict = {}

#create a cluster dict by looping through the dataset and populate the dict with each subset
for clusternum in set(points['cluster']):
    cluster_dict[clusternum] = points[points['cluster']==clusternum].reset_index(drop = True)

In [15]:
def make_cmap(n, name='coolwarm'):
    ## name must be a valid cmap name
    ## returns cmap segmented into n-discrete parts
    return plt.cm.get_cmap(name, n)

cmap = make_cmap(len(cluster_dict),name = 'YlOrRd')

color_dict = {}

for i,code in enumerate(cluster_dict):
    color = cmap(i)
    color_dict[code] = f"RGB({color[0]*256},{color[1]*256},{color[2]*256})"

In [16]:
color_dict

{0: 'RGB(256.0,256.0,204.8)',
 1: 'RGB(255.66535947712418,231.23660130718955,146.57254901960783)',
 2: 'RGB(254.99607843137255,191.74901960784314,90.3529411764706)',
 3: 'RGB(253.9921568627451,141.5529411764706,60.23529411764706)',
 4: 'RGB(244.62222222222223,60.9045751633987,37.479738562091505)',
 5: 'RGB(202.45751633986927,8.700653594771243,34.802614379084964)',
 6: 'RGB(128.50196078431372,0.0,38.14901960784314)'}

In [17]:
#make our own color dict to match steppt branca colormap, explore function later
colordict2 = {0: '#bbf876',1: '#982f36',2: '#a3a3a3',
              3: '#165f97',4: '#79accc',5: '#fc8ee8',
              6: '#583768', 7: '#ffa500'}

In [19]:
m = folium.Map(location = [52.610002, -114.625274], 
                      tiles = 'cartodbpositron', zoom_start = 4, control_scale = True)
m.add_child(steppt)
m.add_child(fullscr)
m.add_child(minimap)

for thing in cluster_dict:
    
    gdf = cluster_dict[thing].fillna("")
    
    def pointcolors(counter):
        return(colordict2[thing])
    
    gdf['color'] = gdf.apply(pointcolors,axis = 1)
    
    marker_cluster = MarkerCluster(control = False, color = 'blue')
    marker_cluster.add_to(m)
    
    sub_group = FeatureGroupSubGroup(marker_cluster, name = thing, control = True, show = True)
    
    for point in range(0, len(gdf)):
        html = """
                <h3>{title}</h3><br>
                <b>Place:</b> {NAME}<br>
                <b>State:</b> {State}<br>
            """
        popup_contents = folium.Html(html.format(title = gdf['cluster'][point],
                                                     NAME = gdf['NAME'][point],
                                                     State = gdf['State'][point]), 
                                                 script = True)
        popup = folium.Popup(popup_contents, max_width=2650)
        folium.vector_layers.CircleMarker(radius = 10,
                                              location = (gdf.geometry[point].y,
                                                          gdf.geometry[point].x),
                                              popup = folium.Popup(popup_contents, max_width=2650),
                                              color = gdf['color'][point],
                                              fill = True,
                                              fill_color = gdf['color'][point],
                                              name = thing,
                                              control = True,
                                              overlay = True
                                             ).add_to(sub_group)
        
    sub_group.add_to(m)
folium.map.LayerControl(collapsed=False).add_to(m)

m

In [32]:
## testing featuregroup
# map = folium.Map(location = [52.610002, -114.625274], 
#                       tiles = 'OpenStreetMap', zoom_start = 4, control_scale = True)



# for i in range(0,len(points)):
#     labels = points['NAME'].iloc[i] + ", " + points['State'].iloc[i] #with the proper for loop this switches values
# for grp_name, df_grp in points.groupby('cluster'):
#     feature_group = folium.FeatureGroup(grp_name)
# for row in df_grp.itertuples():
#     folium.Marker(location=[row.lat, row.lon], popup=labels, icon = icon).add_to(feature_group)
#     feature_group.add_to(map)

# folium.LayerControl().add_to(map)
# map